##### The latest version of this Jupyter notebook is available at [http://github.com/jckantor/CBE40455](http://github.com/jckantor/CBE20255)  for noncommercial use under terms of the [Creative Commons Attribution Noncommericial ShareAlike License](http://creativecommons.org/licenses/by-nc-sa/4.0/).

J.C. Kantor (Kantor.1@nd.edu)

# Binomial Model for an American Call Option

In [1]:
%%script glpsol -m /dev/stdin 

# Binomial Pricing Model for an American Call Option
#
# Determines the price of an American call option using a binomial option
# pricing model. The option price is the minimum value of a portfolio
# that replicates the option payoff at expiration or at early exercise.

# Asset Model
param S0 := 100;         # initial price   
param r := 0.06;         # mean return (annualized)
param sigma := 0.3;      # volatility (annualized)

# Bond Price
param B0 := 1;     uuu7      # initial value
param rf := 0.05;        # risk-free interest rate

# Option
param Kstrike := 100;    # strike price
param Tf := 0.5;           # time to expiration (years)

# Construct a recombining binomial tree
param nPeriods := 10;
set PERIODS := {0..nPeriods};
set STATES {p in PERIODS} := {0..p};
param u := exp(sigma*sqrt(Tf/nPeriods));
param d := 1/u;
param pr := (exp(r*Tf/nPeriods)-d)/(u-d);

param B {p in PERIODS, s in STATES[p]} := B0*(1 + rf*Tf/nPeriods)**p;
param S {p in PERIODS, s in STATES[p]} := S0*(d**(p-s))*(u**(s));

# Replicating Portfolio
var C{p in PERIODS, s in STATES[p]};
var x{p in PERIODS, s in STATES[p]};
var y{p in PERIODS, s in STATES[p]};

portfolio {p in PERIODS, s in STATES[p]}: C[p,s] = x[p,s]*B[p,s] + y[p,s]*S[p,s];

# Self financing constraints
do {p in PERIODS, s in STATES[p] : p < nPeriods}: 
    x[p,s]*B[p+1,s] + y[p,s]*S[p+1,s] >= C[p+1,s];
up {p in PERIODS, s in STATES[p] : p < nPeriods}: 
    x[p,s]*B[p+1,s+1] + y[p,s]*S[p+1,s+1] >= C[p+1,s+1];

# Finance early exercise option
edo{p in PERIODS, s in STATES[p] : p < nPeriods}: 
    x[p,s]*B[p+1,s] + y[p,s]*S[p+1,s] >= S[p+1,s] - Kstrike;
eup{p in PERIODS, s in STATES[p] : p < nPeriods}: 
    x[p,s]*B[p+1,s+1] + y[p,s]*S[p+1,s+1] >= S[p+1,s+1] - Kstrike;
    
# Payoff option value
Payoff {s in STATES[nPeriods]} : C[nPeriods,s] >= max(0, S[nPeriods,s] - Kstrike);

# Objective
minimize OptionPrice : C[0,0];
    
solve;

printf "\n\nOption Price Value = %.2f\n\n\n", C[0,0];
    
table results {p in PERIODS, s in STATES[p]} OUT "CSV" "Binomial_Tree.csv" "Table" :
    p~Period, s~State, B[p,s]~Bond, S[p,s]~Asset, C[p,s]~Option;

end;

GLPSOL: GLPK LP/MIP Solver, v4.52
Parameter(s) specified in the command line:
 -m /dev/stdin
Reading model section from /dev/stdin...
64 lines were read
Generating portfolio...
Generating do...
Generating up...
Generating edo...
Generating eup...
Generating Payoff...
Generating OptionPrice...
Model has been successfully generated
GLPK Simplex Optimizer, v4.52
298 rows, 198 columns, 760 non-zeros
Preprocessing...
274 rows, 175 columns, 712 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.956e+02  ratio =  1.956e+02
GM: min|aij| =  9.669e-01  max|aij| =  1.034e+00  ratio =  1.070e+00
EQ: min|aij| =  9.350e-01  max|aij| =  1.000e+00  ratio =  1.070e+00
Constructing initial basis...
Size of triangular part is 274
      0: obj =   0.000000000e+00  infeas =  1.990e+03 (0)
*    92: obj =   4.887110523e+01  infeas =  4.707e-14 (0)
*   162: obj =   9.426308571e+00  infeas =  1.132e-13 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.7 Mb (691178 bytes)


Option Pr

In [2]:
import pandas
pandas.read_csv("Binomial_Tree.csv")

,Period,State,Bond,Asset,Option
0,0,0,1.000000,100.000000,9.426309
1,1,0,1.002500,93.511848,5.498707
2,1,1,1.002500,106.938321,13.371821
3,2,0,1.005006,87.444657,2.721699
4,2,1,1.005006,100.000000,8.282571
5,2,2,1.005006,114.358044,18.490046
6,3,0,1.007519,81.771115,1.028507
7,3,1,1.007519,93.511848,4.415927
8,3,2,1.007519,106.938321,12.161878
9,3,3,1.007519,122.292572,24.863523
